<a href="https://colab.research.google.com/github/OrensteinLab/PrimerDesigner/blob/main/Tracking_Timings_Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pre-Calculations

##Installs & Imports

In [ ]:
!pip install -U bokeh seaborn pandas
!pip install primer3-py biopython pandarallel
!pip install pulp
!pip install gurobipy
!pip install biopython==1.75




  Cloning https://github.com/FordyceLab/seequence.git to /tmp/pip-install-gum8rkv0/seequence_7b33976eaff540238436abf8da4ba80d
  Running command git clone --filter=blob:none --quiet https://github.com/FordyceLab/seequence.git /tmp/pip-install-gum8rkv0/seequence_7b33976eaff540238436abf8da4ba80d
  Resolved https://github.com/FordyceLab/seequence.git to commit 3ea730537fcf5b7ef807ebf6e057f5bf4e875bb9
  Preparing metadata (setup.py) ... done


In [ ]:
# IMPORTS
import time

import random as rand
from itertools import product

from pandarallel import pandarallel as pl
pl.initialize()

import primer3 as p3
from Bio.Seq import Seq

import itertools as it
import numpy as np
import pandas as pd
pd.set_option('display.precision', 1)
import networkx as nx

import gurobipy as gp

import tracemalloc
import matplotlib.pyplot as plt

def revcomp(seq):
  return str(Seq(seq).reverse_complement())
def translate(seq):
  return str(Seq(seq).translate())

print('Ready')



## Create Primer sequences

In [ ]:
def subsequences(sequence, lmin, lmax): #Generates all subsequences w/ all poss. start-stop pairs
  ls = []
  for j in range(lmin, lmax+1): #length
    for i in range(len(sequence)-j+1): #starting index
      start = i
      stop = i+j
      ls.append([sequence[start:stop], start, stop, stop-start])
  return pd.DataFrame(ls, columns=['seq','start','stop','len'])

## Primer Data
##### Creates the primer pandas dataframe containing information about every possible primer in the protein coding sequence. The function calculates various primer parameters including gc content, delta G and melting temepratures. The cost is also calculated for every primer

In [ ]:
def create_primer_df():
  # convention: start index of r-primers will be 3' (i.e. start < stop)
  primer_f = pd.DataFrame(columns=['seq','start','stop','fr','len'])
  primer_f[['seq','start','stop','len']] = subsequences(sequence_nt, primer_lmin, primer_lmax)
  primer_f['fr'] = 'f'

  #Shifting so that 0 is at the start of mutreg (upstream has negative values)
  primer_f['start'] = primer_f.start - mutreg_start
  primer_f['stop'] = primer_f.stop - mutreg_start

  #Creating reverse primers at same locations
  primer_r = primer_f[['seq','start','stop','fr','len']].copy()
  primer_r['fr'] = 'r'
  primer_r['seq'] = primer_r.seq.apply(revcomp)

  #Concatenating Forward & Reverse
  primer_df = pd.concat([primer_f,primer_r])
  primer_df.sort_values(by=['start','stop','fr'], inplace=True)

  #Calculating "Cost" Values
  primer_df['gc'] = primer_df.seq.apply(GC)
  primer_df['tm'] = primer_df.seq.apply(pcr.calcTm)
  res = primer_df.seq.parallel_apply(lambda s: pcr.calcHairpin(s).todict())
  primer_df['hp_tm'] = res.apply(lambda res: res['tm'])
  primer_df['hp_dg'] = res.apply(lambda res: res['dg']*1e-3)
  res = primer_df.seq.parallel_apply(lambda s: pcr.calcHomodimer(s).todict())
  primer_df['ho_tm'] = res.apply(lambda res: res['tm'])
  primer_df['ho_dg'] = res.apply(lambda res: res['dg']*1e-3)

  def primer_cost(primer):
    # calculates primer cost based on homodimer an haipin delta G, tm cost, and len cost
    hp_dg_max = -4
    ho_dg_max = -8
    tm_min = 58

    tm_cost = max(0, tm_min-primer.tm)**1.7
    hp_cost = max(0, hp_dg_max - primer.hp_dg)**1.2
    ho_cost = max(0, ho_dg_max - primer.ho_dg)**1.2
    len_cost = primer.len*1e-5

    # add high cost if there is a 4-mer of the nucleotide
    sequence = primer.seq
    for nucleotide in "ATGC":
        if nucleotide * 4 in sequence:
            cost += 1000000
            break

    cost = hp_cost + ho_cost + len_cost + tm_cost
    return cost

  primer_df['cost'] = primer_df.parallel_apply(primer_cost, axis=1)
  primer_df['log10cost'] = primer_df.cost.apply(np.log10)

  primer_df.reset_index(inplace=True)
  primer_f = primer_df.query('fr=="f"').reset_index(drop=True)
  primer_r = primer_df.query('fr=="r"').reset_index(drop=True)
  primer_df.set_index(['start','stop','fr'], inplace=True)

  return primer_f, primer_df

def check_threshold(tm,gc):
  # returns false only if the threshold flag is on and primers did not pass threshold
  if not apply_threshold:
    return True
  else:
    if min_gc <= gc <= max_gc and  min_tm <= tm <=max_tm:
      return True
    else:
      return False



## Primer Class
##### Creates a primer class which is used for the creation of the primer graph

In [ ]:
class Primer:
  def __init__(self,start,stop,is_r=False):
    assert start < stop
    self.start = start
    self.stop = stop
    self.is_r = is_r #forward or reverse
    self.l = stop-start #length
    self.w = primer_df.at[self.tup(),'cost'] #total cost value; the fancy notation is b/c
                                             #of the hierarchal lookup system in panda.df

  def __str__(self):
    return ' '.join(map(str,(self.start, self.stop, self.is_r)))
  def __repr__(self):
    return f'{("r" if self.is_r else"f")}({self.start},{self.stop})'
  def tup(self):
    return (self.start,self.stop,("r" if self.is_r else"f"))


def actions(primer): #returning possible counterparts (forward -> reverse; reverse -> forward)
                     #i.e. this method gets the "neighbors"
  if not primer.is_r:  # fwd
    for oligo_l, primer_l in it.product(reversed(range(oligo_lmin, oligo_lmax+1)),
                                        range(primer_lmin,primer_lmax+1)):

      stop = primer.start + oligo_l
      start = stop - primer_l

      if apply_threshold:
        # finds tm of forward and reverse primers
        tm_f=primer_df.at[primer.tup(),'tm']
        tm_r=primer_df.at[(start,stop,"r"),'tm']

        # if tm difference is larger then max_difference threshold do not add primer to graph
        if abs(tm_f-tm_r)>max_difference:
          continue

      yield Primer(start, stop, is_r=True)

  elif primer.is_r:  # rev
    for overlap_l, primer_l in it.product(reversed(range(overlap_lmin, overlap_lmax+1)),
                                          range(primer_lmin,primer_lmax+1)):

      start = primer.stop - overlap_l
      stop = start + primer_l


      # filter
      no_split = (primer.start - stop) >= primer.start%3
      if (stop > primer.start) or (not no_split): ## redundant to check first condition?
        continue
      yield Primer(start, stop)

## Graph DFS
##### Recursive function used for creating the primer graph representing all possible forward and reverse primer combinations

In [ ]:
def dfs(primer): #CREATING the graph

  tm=primer_df.at[primer.tup(),'tm']
  gc=primer_df.at[primer.tup(),'gc']

  if (primer.start >= mutreg_l) and primer.is_r and check_threshold(tm,gc) :  # base case (end)
    G.add_edge(primer.tup(),'d', weight=0.) #G is global variable defined in next section
    return

  for next_primer in actions(primer):

    is_new = not G.has_node(next_primer.tup()) # check if primer node exists already

    tm=primer_df.at[next_primer.tup(),'tm']
    gc=primer_df.at[next_primer.tup(),'gc']

    passes_threshold= check_threshold(tm,gc) # check if primer passes threshold

    # only add edge if primer passes threshold
    if passes_threshold:
      G.add_edge(primer.tup(),next_primer.tup(), weight=next_primer.w) #weight is the cost of the new primer
    if passes_threshold and is_new:
      dfs(next_primer)

def paths_ct(G, u, d): #total # of paths between two points
    if u == d:
        return 1
    else:
        if not G.nodes[u]: #npaths attribute is the # of paths out of u
            G.nodes[u]['npaths'] = sum(paths_ct(G, c, d) for c in G.successors(u))
        return G.nodes[u]['npaths']

## Greedy Algorithm
##### Runs the baseline greedy algorithm which is compared to PrimerDesigner

In [ ]:
def run_greedy(G, primer_f, primer_df):
  path_ls = [[]]
  G_sub = G.copy()
  for i in range(num_proteins):
    nodes_to_remove = []
    for p,n in it.product(path_ls[-1],G_sub.nodes()):
      if n=='s' or n=='d':
        continue
      pn_intersect = n[1]-p[0] > allowed_overlap and p[1]-n[0] > allowed_overlap and n[2]==p[2]  ## check overlap
      if pn_intersect:
        nodes_to_remove.append(n)
    G_sub.remove_nodes_from(set(nodes_to_remove))
    try:
      path_ls.append([primer for primer in nx.algorithms.shortest_path(G_sub,'s','d', weight='weight')][1:-1])
    except:
      print(f'WARNING: No feasible primer sequence for lib_{i}; reduce number of libraries or relax constraints.')

  path_ls = path_ls[1:]

  # # https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
  primer_set = pd.DataFrame()
  for i,primer_ls in enumerate(path_ls):
    primer_set1 = primer_df.loc[primer_ls].copy().reset_index()
    primer_set1['lib_i'] = i
    primer_set = pd.concat([primer_set, primer_set1])
  primer_set['tile_i'] = primer_set.groupby(['lib_i','fr']).cumcount()
  primer_set = primer_set[['lib_i','tile_i']+primer_set.columns[:-2].to_list()]

  #print(primer_set.groupby('lib_i').cost.sum())
  cost = primer_set.cost.sum()

  return path_ls, cost

##ILP Functions

## Gurobi Setup
##### The user should input their Gurobi license details for the ILP part

In [ ]:
def get_model():
  params = {
  "WLSACCESSID":"",
  "WLSSECRET":"",
  "LICENSEID":
  }
  env = gp.Env(params=params)

  # Create the model within the Gurobi environment
  model = gp.Model('min-sum', env=env)
  return model

## Model Creation and Run

##### This function creates the ILP solver and represents the primer graph as an ILP optimization problem. It adds the ILP contraints based on overlap and single path contraints. Then, it runs the ILP solver and returns primer path solution

In [ ]:


def get_info(graphs):
  setup_start = time.time()
  tracemalloc.start()
  #Create Model
  model = get_model()
  #Getting Nodes/Edges
  graph_edges = G.edges(data=True)
  graph_nodes = [node for node in G.nodes if node != 's' and node != 'd'] #removing s & d nodes

  #for G in graphs:
   # graphs_edges.append(G.edges(data=True))
    #graphs_nodes.append(sorted((node for node in G.nodes if node != 's' and node != 'd'), key = lambda x:x[0:2])) #removing s & d

  def create_bins(): # bins take the form (start, end, [])
    all_bins = {(start, start+len, c):[] for start in range(*nt_range) for len in range(*l_range) for c in ('f', 'r')}

    for node in graph_nodes: #for each node, add it into all the necessary bins
      for bin_start in range(node[0], node[1]):
        for bin_length in range(*l_range):
          if bin_start + bin_length > node[1]:
            break
          else:
            all_bins[(bin_start, bin_start + bin_length, node[2])].append(node)

    # all_bins = {key:val for key,val in all_bins.items() if val} #no empty bins

    return all_bins

  all_bins = create_bins()
  print("Number of Constraints:", len(all_bins))
  print("Average Vars Per Constraint", 1/len(all_bins) * sum(len(val) for _, val in all_bins.items()))

  def unite_bins():
    # this function merges all bins corresponding to identical sequences
    united_bins={}

    for bin in all_bins.keys():
      start=bin[0]
      end=bin[1]
      fr=bin[2]
      length=end-start
      bin_sequence = sequence_nt[start+mutreg_start:end+mutreg_start]
      if (bin_sequence,fr) in united_bins:
        continue
      bin_union=[]
      # find subsequences that are identical to the bin sequence and add them the the union of bins
      for i in range(len(sequence_nt)-length+1):
        if sequence_nt[i:i+length] == bin_sequence:
          start = i - mutreg_start # subtract from start to account for upstream region
          bin_union.extend(all_bins[(start, start + length,fr)]) # add the bins with identical sequences to the union list

      # add bin union tp dictionary in the sequence entry
      united_bins[(bin_sequence,fr)]=bin_union

    return united_bins


  # if the merge_bins flag is set to true, calls function to unite bins with identical sequences
  if merge_bins:
    united_bins= unite_bins()
    print("Number of united bins constraints:", len(united_bins))
    print("Average Vars Per Constraint", 1/len(united_bins) * sum(len(ls) for seq,ls in united_bins.items()))
    all_bins= united_bins


  #Converting Graphs to Lists
  ij = gp.tuplelist()
  w_ij = gp.tupledict()

  for edge in graph_edges:
    l = (str(edge[0]), str(edge[1])) #i, j
    ij.append(l)
    w_ij[l] = edge[-1]['weight']

  print("Finished Conversion")

  #Adding Variables
  x = model.addVars(ij, obj=w_ij, vtype=gp.GRB.BINARY)
  print("Finished Variable Creations")

  # Intersection Constraints - bins
  for cnt, nodes in enumerate(all_bins.values()):
    all_edges = []
    if cnt % (len(all_bins)//25) == 0:
          print(int(cnt / len(all_bins) * 100))
    for node in nodes:
      all_edges.append(x.sum(str(node), '*'))
    model.addConstr(gp.quicksum(all_edges) <= 1)
  print("Finished Intersection constraints!")

  #Single Path Constraints
  for n in graph_nodes + ['s', 'd']: #adding s & d back just here
    v = str(n)
    model.addConstr(sum(x[i,j] for i,j in ij.select(v, '*')) - sum(x[j,i] for j,i in ij.select('*', v)) == (num_proteins if v=='s' else -1 * num_proteins if v=='d' else 0), v)

  setup_time = time.time() - setup_start
  setup_memory = tracemalloc.get_traced_memory()[1] / 10**6
  tracemalloc.stop()

  tracemalloc.start()
  start_time = time.time()
  model.optimize()
  ILP_time = time.time() - start_time
  ILP_memory = tracemalloc.get_traced_memory()[1] / 10**6
  tracemalloc.stop()

  def post_processing(variables):
    all_proteins = [['s'] for _ in range(num_proteins)]
    true_edges = [index for index, var in x.items() if var.X != 0]

    while true_edges:
      edge = true_edges.pop(0)
      added_edge = False
      for protein_list in all_proteins:
        if edge[0] == protein_list[-1]:
          protein_list.append(edge[1])
          added_edge = True
          break
      if not added_edge:
        true_edges.append(edge)

    return all_proteins

  actual_values = post_processing(x)
  for cnt, vals in enumerate(actual_values):
    print(f"Protein #{cnt+1} ({len(vals)})")
    print(vals)
    print()

  return model.numVars, model.numConstrs, setup_time, setup_memory, ILP_time, ILP_memory, actual_values, model.objVal

##Full Sequence
##### The user can choose the mutreg sequence and upstream and downstream regions

In [ ]:
# SpAP protein
upstream_nt = 'GCTAGTGGTGCTAGCCCCGCGAAATTAATACGACTCACTATAGGGTCTAGAAATAATTTTGTTTAACTTTAAGAAGGAGATATACAT'
mutreg_nt_full='ATGCAAAGCCCAGCACCTGCCGCAGCGCCTGCCCCTGCGGCACGTTCCATCGCAGCTACGCCTCCTAAACTGATCGTGGCAATTAGCGTGGACCAGTTTAGTGCAGACTTGTTCTCGGAGTATCGTCAATATTACACCGGAGGTTTAAAGCGTCTTACATCCGAAGGAGCTGTGTTCCCACGTGGTTATCAGAGTCATGCGGCAACAGAAACGTGTCCTGGTCACTCAACGATCCTGACAGGATCACGTCCGTCACGTACGGGTATTATCGCTAATAACTGGTTCGACTTGGACGCAAAGCGTGAGGATAAAAATCTGTACTGTGCTGAGGATGAATCCCAACCCGGTAGTTCGTCTGACAAGTACGAAGCTTCGCCACTGCACTTAAAGGTACCCACCCTGGGGGGACGCATGAAAGCCGCCAATCCTGCGACTCGTGTCGTCTCTGTTGCCGGCAAGGATCGCGCGGCCATTATGATGGGTGGCGCCACAGCGGATCAGGTCTGGTGGTTAGGGGGGCCTCAGGGGTATGTTTCGTATAAGGGTGTAGCGCCAACTCCCCTTGTAACACAGGTCAATCAGGCCTTTGCACAGCGCTTAGCTCAGCCGAACCCGGGATTTGAGTTGCCTGCTCAGTGCGTCAGCAAGGACTTTCCTGTTCAAGCGGGAAATCGCACAGTGGGTACCGGCCGCTTCGCCCGTGATGCTGGTGACTACAAAGGTTTTCGCATTTCCCCGGAGCAGGATGCTATGACGCTTGCATTCGCTGCCGCGGCCATTGAAAATATGCAATTAGGGAAGCAGGCCCAGACCGATATTATTAGCATTGGACTGAGCGCTACGGATTACGTGGGACACACCTTCGGCACGGAGGGTACGGAGAGTTGCATCCAAGTGGATCGTTTAGACACGGAGCTTGGTGCATTCTTTGATAAACTGGATAAGGATGGGATTGACTACGTAGTAGTGCTGACTGCAGATCATGGAGGACACGATCTGCCCGAACGTCATCGTATGAATGCCATGCCGATGGAACAGCGCGTAGACATGGCCCTGACACCTAAAGCTCTGAATGCTACCATCGCTGAGAAAGCTGGCCTTCCGGGCAAAAAGGTTATTTGGTCAGATGGACCTTCTGGCGATATTTACTATGATAAGGGCCTTACAGCCGCTCAACGTGCCCGTGTTGAAACCGAGGCGTTAAAATACTTGCGCGCGCATCCCCAAGTACAGACTGTATTCACTAAGGCGGAAATCGCGGCTACCCCTTCTCCGTCGGGACCACCTGAGAGCTGGAGTTTGATCCAGGAAGCTCGCGCGTCATTTTACCCGTCGCGCTCCGGGGACCTGTTACTTTTATTGAAACCTCGTGTGATGAGCATTCCTGAGCAAGCAGTCATGGGCTCGGTTGCAACCCATGGATCTCCATGGGATACGGATCGCCGTGTGCCTATCCTGTTTTGGCGCAAAGGTATGCAGCATTTCGAACAACCCTTAGGAGTAGAGACTGTTGATATTTTGCCCTCCTTGGCTGCACTTATTAAGCTTCCTGTTCCTAAGGATCAGATCGACGGCCGCTGTCTGGACTTGGTCGCCGGCAAGGATGATTCCTGTGCTGGACAGGGA'
downstream_nt = 'GGAGGGTCTGGGGGAGGAGGCAGTGGCATGGTGAGCAAGGGCGAGGAGCTGTTCACCGGGGTGGTGCCCATCCTGGTCGAGCTGGACGGCGACGTAAACGGCCACAAGTTCAGCGTGTCCGGCGAGGGCGAGGGCGATGCCACCTACGGCAAGCTGACCCTGAAGTTCATCTGCACCACCGGCAAGCTGCCCG'

##Parameters
##### The user can adjust algorithm parameters such as primer length, oligo length and allowed overlap

In [ ]:
# adjust algorithm parameters
primer_lmin, primer_lmax = 18, 30
overlap_lmin,overlap_lmax = 45,50
oligo_lmin,oligo_lmax = 195,205
num_proteins = 3
allowed_overlap = 6

# by setting the unite bins to true, the user can choose to unite bins with identical sequences
merge_bins=True

# sets up pcr reactions with parameters
pcr = p3.thermoanalysis.ThermoAnalysis(dna_conc= 250,
                                       mv_conc= 50,
                                       dv_conc= 0,
                                       dntp_conc= 0,
                                       tm_method= 'santalucia',
                                       salt_correction_method= 'owczarzy',
                                       temp_c= 25)

## Efficiency Thresholds
##### The user can choose to apply primer efficiency threshold on gc content and tm and the maximum allowed tm different between forward and reverse primer in each pair

In [ ]:

apply_threshold= False # apply threshold flag. If true, threshold will be applied
min_gc=40
max_gc=60
min_tm=58
max_tm=65
max_difference=3

##Code To Run
##### Runs PrimerDesigner on different sequence lengths. Records results in a dictionary which is written to a csv file

In [ ]:
all_data = []

for seq_length in range(226,1626,100):

  mutreg_nt=mutreg_nt_full[:seq_length]
  sequence_nt = upstream_nt + mutreg_nt + downstream_nt
  mutreg_l = len(mutreg_nt)
  mutreg_start = len(upstream_nt)
  mutreg_stop = mutreg_start + mutreg_l
  mutreg_aa = translate(mutreg_nt)

  nt_range = (-len(upstream_nt), len(mutreg_nt) + len(downstream_nt)+1) #range of nucleotides
  l_range = (allowed_overlap+1, primer_lmax+1)

  mut_df = create_mut_df()
  primer_f, primer_df = create_primer_df()

  #Creating the Graph
  start_time = time.time()
  tracemalloc.start()
  G = nx.DiGraph()
  primers_init = [Primer(p.start,p.stop) for _,p in primer_f.query('stop<=0').iterrows() if check_threshold(p.tm,p.gc)]  ## all forward primers that end before mutreg and pass threshold
  for primer in primers_init:
    G.add_edge('s',primer.tup(), weight=primer.w) #intializing the s-primer connection
    dfs(primer) #create the rest of the graph

  graph_time = int(time.time() - start_time)
  graph_memory = tracemalloc.get_traced_memory()[1] / 10**6 #MB
  tracemalloc.stop()

  #Greedy Solution
  start_time = time.time()
  greedy_solution, greedy_obj = run_greedy(G, primer_f, primer_df)
  greedy_time = time.time() - start_time

  #Convert to ILP & Solve
  tracemalloc.start()
  start_time = time.time()

  numVars, numConstrs, setup_time, setup_memory, ILP_time, ILP_memory, actual_values, objective = get_info([G, G])
  #numVars, numConstrs, setup_time, setup_memory, ILP_time, ILP_memory = 0,0,0,0,0,0
  all_data.append({"seq length:": seq_length,
                    "Nodes":len(G.nodes),
                    "Edges": len(G.edges),
                    "Time (Graph)": graph_time,
                    "MP (Graph)": graph_memory,
                    "Vars": numVars,
                    "Constr": numConstrs,
                    "Time (Setup)": setup_time,
                    "MP (Setup)": setup_memory,
                    "Time (ILP)": ILP_time,
                    "MP (ILP)": ILP_memory,
                    "ILP Solution": actual_values,
                    "ILP Objective": objective,
                    "Greedy Solution": greedy_solution,
                    "Greedy Objective": greedy_obj,
                    "Greedy Time": greedy_time})
  print(all_data[-1])

In [ ]:
import pandas as pd

# Assuming all_data is a list of dictionaries
df = pd.DataFrame(all_data)

# Write the DataFrame to a CSV file
df.to_csv('data.csv', index=False)

